# Control flow

## Topics
- conditional statements
- repeated evaluation
    - `while` and `for`
- breaking and continuing 

See also the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/
)

## Numeric Comparisons

Standard comparison operations are defined for all the primitive numeric types:


- `==`: equality 
- `!=`, `≠`: inequality             
- `<`: less than              
- `<=`, `≤`: less than or equal to   
- `>`: greater than     
- `>=`, `≥`: greater than or equal to

Conditions can be chained, i.e., `0 < x < 1`

## Conditional statements

The most common conditional statement is the `if` statement. Syntax is
```julia
if *condition*
    *option 1*
elseif *condition 2*
    *option 2*
else
    *option 3*
end
```

In [ ]:
variable = 6
if variable > 5
    println("Wow! The variable is larger than 5")
elseif variable > 0
    println("Well, it is positive, at least")
else
    println("Phew, too small number!")
end

Julia is relying on the `end` keyword to close the statements!

## And, or, ...
- And is expressed as `&&`, i.e., `*condition1* && *condition2*`
- Or is expressed as `||`, i.e., `*condition1* || *condition2*`


In [ ]:
false && true

In [ ]:
false || true

## Loops

### For loops

We will want to update all the cells, so we will need a loop structure. The for loop will run a set of commands with each cell
in an array or anything else iterable.

The syntax is
```julia
for *var* in *loop iterable*
    *loop body*
end
```

In [ ]:
for n in 1:10
    println(n)
end

#### Iterating over a list
For loops can also be used to iterate over containers

In [ ]:
for i in [1,4,0]
    println(i)
end

In [ ]:
for s ∈ ["foo","bar","baz"]
    println(s)
end

#### Enumerate: A life saver
Iterate over an array with index AND value. 

Syntax is
```julia
for (index, value) in enumerate(arr)
    println((index,value))
end
```

In [ ]:
for (i,s) ∈ enumerate(["foo","bar","baz"])
    println(i, " ", s)
end

## Breaking and continuing
Sometimes you need to terminate a `while` or `for` evaluation before the end.  This can be accomplished with the `break` keyword


In [ ]:
i = 1;
while true
    println(i)
    if i >= 5
        break
    end
    i += 1
end

In [ ]:
for i = 1:1000
    println(i)
    if i >= 5
        break
    end
end

In other circumstances, it is handy to be able to stop an iteration and move on to the next one immediately. The `continue` keyword accomplishes this.


In [ ]:
for i = 1:10
    if i % 3 != 0
        continue
    end
    println(i)
end

## The simulation

In [83]:
# So we can use this to update the epidemic simulation.

cells = Int8[0 0 0 0 0 ;
             0 0 0 0 0 ;
             0 0 1 0 0 ;
             0 0 0 0 0 ;
             0 0 0 0 0 ]

5×5 Matrix{Int8}:
 0  0  0  0  0
 0  0  0  0  0
 0  0  1  0  0
 0  0  0  0  0
 0  0  0  0  0

Let's first write the update for a single cell. We need to loop over each of it's neighbours and see if it infects the cell.

In [93]:
# Check the neighbours of the cell at 2, 3
for (nb_i, nb_j) in [(1,3), (3,3), (2,2), (2,4)]
    
    # Check if the cell is infected
    if cells[nb_i, nb_j] == 1
        
        # Infect with probability 0.1
        if rand(1)[1] < 0.1
            new_cells[i,j] = 1
        end
    end
end

Now the whole update loop. We will loop over each cell in the 2D array and do the same for each.

Notice that we want to change the value of the array. The notation for this is `cells[i,j] = new_value`.

In [107]:
# Create a copy to hold the new state
new_cells = deepcopy(cells)

# loop over rows
for i in 1:size(cells)[1]
    # and over columns
    for j in 1:size(cells)[2]
        
        # Check each neighbour for infected cells
        for (nb_i, nb_j) in [(i-1,j), (i+1,j), (i,j-1), (i,j+1)]
            
            # Handle boundaries. On boundary cells, some neighbours don't exist.
            if nb_i > size(cells)[1] || nb_i < 1
                continue
            end
            
            if nb_j > size(cells)[2] || nb_j < 1
                continue
            end
            
            if cells[nb_i, nb_j] == 1
                # Infect with probability 0.1
                if rand(1)[1] < 0.1
                    new_cells[i,j] = 1
                end
            end
            
        end
        
    end
end

cells = new_cells

5×5 Matrix{Int8}:
 0  0  0  1  0
 0  1  1  1  1
 0  0  1  1  1
 0  1  1  1  1
 1  1  1  0  1

We could do multiple updates in a for loop, but the code is already getting a bit clunky. The large number of nested for loops makes it
hard to read. Instead, we will first split it into functions in the next section.

Functions are an especially important topic in Julia. They are the compilation unit of the JIT compiler. Writing code into functions makes it more modular and easier to handle, but it also makes it faster.

## Nested loops
Multiple nested loops can be combined into a single outer loop. 

In [ ]:
for i = 1:2
    for j = 3:3
        println((i,j))
    end
end

Translates into:

In [ ]:
for i = 1:2, j = 3:4
    println((i, j))
end

A `break` statement inside such a loop exits the entire nest of loops, not just the inner one.


## While statement
Syntax is 
```julia
while *condition*
    *loop body*
end
```

In [ ]:
n = 0
while n < 10
    n += 1
    println(n)
end

## List comprehension
Comprehensions provide a general and powerful way to construct arrays. 

Comprehension syntax is similar to set construction notation in mathematics
```julia
A = [ F(x,y,...) for x=rx, y=ry, ... ]
```

In [ ]:
[(i,j) for i=1:2 for j=1:i]

## Ternary operator
Even though the name for this operation is scary, it is actually very easy to understand and handy to use. Syntax is:
```julia
*condition* ? *do 1* : *do 2*
```
which is equal to writing
```julia
if *condition*
    *do 1*
else
    *do 2*
end
```

In [ ]:
# What does the followning code do? Try it out by giving values to `x` and `y`
x = 
y = 
(x > y ) ? x : y

## Advanced: Tasks (aka Coroutines)
Tasks are a control flow feature that allows computations to be suspended and resumed in a flexible manner. More information can be found from the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/#man-tasks-1)

In [ ]:
function producer(c::Channel)
    put!(c, "start")
    for n=1:4
        put!(c, 2n)
    end
    put!(c, "stop")
end;

chnl = Channel(producer);

In [ ]:
take!(chnl) # try executing me repeatedly